## Cleaning Up Lots of Data

In a previous notebook regarding the data from the World Happiness Report, I cleaned up data - added rows, imputed values - for a single region, only 9 countries. It wasn't challenging, although it was time consuming. If I had done this for multiple regions, or the entire dataset, it would get pretty redundant. Can I simplify the process?

In the time since I started playing with the data, researchers released the [https://worldhappiness.report/ed/2019/](2019 World Happiness Report). Hooray! More data to play with! So, the data I'm using here is similar, but not the same, as my other notebooks.

Why am I still working with this data? Well, I've been playing around with Tableau recently, and I need a good dataset to visualize. I really like this data, so I need to clean it all up so that I can visualize it better.

[Note: If you want to see my visualizations, check them out [on Tableau Public](https://public.tableau.com/profile/natalie.denning#!/vizhome/NEW-global-happiness/WorldHappinessReport)!]

### The Process

Well, what do I need to do to turn this into good, clean data?

- Remove unwanted columns and rows from year 2005
- Create new df for a single country
- Find whether country is missing rows/years
- Add blank rows corresponding to missing years
- Sort rows by year
- Interpolate, ffill, bfill
- Smash back together.

In [1]:
import pandas as pd
import numpy as np

In [6]:
happy = pd.read_excel('world-happiness-2019.xls')

In [7]:
# Which columns do I want to keep?

columns_to_keep = ['country', 'year', 'Life Ladder',
       'Log GDP per capita', 'Social support',
       'Healthy life expectancy at birth', 'Freedom to make life choices',
       'Generosity', 'Perceptions of corruption', 'Positive affect',
       'Negative affect', 'Confidence in national government',
       'Democratic Quality', 'Delivery Quality']

In [22]:
# Remove rows from the year 2005, remove unwanted columns

happy_2 = happy[happy['year'] != 2005].filter(columns_to_keep)

In [11]:
# Fill missing values with interpolate, bfill, and ffill
# Interpolate is capped at 3 - if more than 3 null values in neighboring years, just return nulls

def fill_missing_values(df):
    filled_df = df.sort_values('year').interpolate(limit=3).fillna(method='bfill').fillna(method='ffill').reset_index().drop('index', axis=1)
    return filled_df

In [17]:
# Fill missing data (rows and values) for single-country df

def filled_df_single(df):
    
    # If all 12 years/rows present for country..
    if len(df['year']) == 12: 
        # Fill missing values only
        return fill_missing_values(df)
    
    # If fewer than 6 rows..
    elif len(df['year']) < 6:
        # Don't add new rows; fill null values only
        return fill_missing_values(df)
    
    # If country has 6-11 rows in df...
    else:
        # Find missing years/rows
        a = df['year'].astype(int).tolist()
        b = [x for x in range(2006, 2019)]
        a = set(a)
        missing_years = list((a^set(b)))
        
        # Add new rows, fill with null values
        for num in range(len(missing_years)):
            df.loc[df.index.max() + 1 + num] = np.nan
        
        # Add country name to new rows
        country_name = df['country'][0]
        df['country'] = country_name
        
        # Get indices for null rows
        new_series = df['year'].apply(lambda x: x != x)
        null_indices = list(new_series[new_series == True].index)
        
        # Fillna with missing years at null indices
        replacement_tuples = list(zip(null_indices, missing_years))
        replacement_dict = dict(replacement_tuples)
        df['year'].fillna(value = replacement_dict, inplace=True)
        
        # Fill missing values
        return fill_missing_values(df)

In [14]:
# Function for multi-country df
# Split off country, run through function, append to new dataframe

def filled_df_multi(df):
    # Empty list to append finished dfs
    df_list = []

    for country in df['country'].unique():
        
        # If there's only one row, this will return a Series instead of a df
        if 'year' in df.set_index('country').loc[country].index:
            # So we need to just turn it into a single-row df before adding it to empty df
            df2 = df.set_index('country').loc[country].to_frame().transpose().reset_index()
        
        # Otherwise, create single-country dataframe and run through function
        else:
            df2 = df.set_index('country').loc[country].reset_index()
        df3 = filled_df_single(df2)
        df_list.append(df3)

        full_df = pd.concat(df_list, ignore_index=True, sort=False)
        
    # Don't want a new "index" column
    if 'index' in full_df:
        return full_df.drop('index', axis=1)
    else:
        return full_df

In [19]:
happy_3 = filled_df_multi(happy_2)

In [15]:
# Compare new df to old df

happy_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1677 entries, 0 to 1703
Data columns (total 14 columns):
country                              1677 non-null object
year                                 1677 non-null int64
Life Ladder                          1677 non-null float64
Log GDP per capita                   1649 non-null float64
Social support                       1664 non-null float64
Healthy life expectancy at birth     1649 non-null float64
Freedom to make life choices         1650 non-null float64
Generosity                           1621 non-null float64
Perceptions of corruption            1583 non-null float64
Positive affect                      1659 non-null float64
Negative affect                      1665 non-null float64
Confidence in national government    1506 non-null float64
Democratic Quality                   1531 non-null float64
Delivery Quality                     1532 non-null float64
dtypes: float64(12), int64(1), object(1)
memory usage: 196.5+ KB


In [23]:
# Compare new df to old df

happy_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1884 entries, 0 to 1883
Data columns (total 15 columns):
country                              1880 non-null object
year                                 1884 non-null float64
Life Ladder                          1884 non-null float64
Log GDP per capita                   1864 non-null float64
Social support                       1883 non-null float64
Healthy life expectancy at birth     1865 non-null float64
Freedom to make life choices         1884 non-null float64
Generosity                           1863 non-null float64
Perceptions of corruption            1856 non-null float64
Positive affect                      1883 non-null float64
Negative affect                      1884 non-null float64
Confidence in national government    1783 non-null float64
Democratic Quality                   1867 non-null float64
Delivery Quality                     1867 non-null float64
level_0                              4 non-null float64
dtypes: floa

Done! This function added about 200 new rows. While not all of the null values are gone (that would have been impossible), they are significantly reduced.

In [21]:
happy_3.to_csv('2019_happiness_full.csv')